In [1]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# get some old helper func
def connect_my_db(secrets):
    """
    connect to the database
    :param secrets: path of where the connection string is
    :return: db connection (engine from sqlalchemy
    """
    with open(secrets) as file:
        uri = yaml.load(file, Loader = yaml.FullLoader)
    uri = uri["db_string"]

    return create_engine(uri)

def read_table(engine, table_name, date_col = 'YEAR_MONTH', date_start = None, date_end = None):
    """
    Read the table from PostgreSQL
    :param engine: the connection engine from sqlalchemy
    :param table_name: name of the table
    :param date_start: start date the data to retrieve
    :param date_end: end date the data to retrieve
    :return: the data frame read from the PostgreSQL
    """
    if None not in (date_start, date_end):
        query = 'SELECT * FROM ' + f'"{table_name}" WHERE "{date_col}" between '+ f"'{date_start}' and '{date_end}'"
    else:
        query = 'SELECT * FROM ' + f'"{table_name}"'

    df = pd.read_sql(query, engine)
    return df

In [3]:
# get access to db
%cd C:\Users\tyler\repos\airbnb-forecast
engine = connect_my_db("secrets/db_string")

C:\Users\tyler\repos\airbnb-forecast


In [4]:
# get data
df_list = read_table(engine, "FS_LIST_MONTHLY")
df_cal = read_table(engine, "FS_CAL")
df_host = read_table(engine, "FS_HOST_M")
df_rev = read_table(engine, "FS_REVIEW_MONTHLY")
df_booked = read_table(engine, "FS_BOOKED_MONTHLY")
df_loc = read_table(engine, "FS_LOCATION_MONTHLY")
df_time = read_table(engine, "FS_TIME_MONTHLY")
df_price = read_table(engine, "FS_PRICE_MONTHLY")

# Merge the features together to become main dataset
df_data = df_list.merge(df_cal, on = ['ID','YEAR_MONTH']).\
    merge(df_host, on = ['ID','YEAR_MONTH']).\
    merge(df_rev, on = ['ID','YEAR_MONTH']).\
    merge(df_booked, on = ['ID','YEAR_MONTH']).\
    merge(df_loc, on = ['ID','YEAR_MONTH']).\
    merge(df_time, on = ['ID','YEAR_MONTH']).\
    merge(df_price, on = ['ID','YEAR_MONTH'])

In [5]:
df_data.YEAR_MONTH.unique()

array(['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06'],
      dtype=object)

In [6]:
df_data.columns

Index(['ID', 'YEAR_MONTH', 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES',
       'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'AMENITIES', 'SQUARE_FEET',
       'INSTANT_BOOKABLE', 'IS_BUSINESS_TRAVEL_READY', 'GUESTS_INCLUDED',
       'CANCELLATION_POLICY', 'REQUIRE_GUEST_PROFILE_PICTURE',
       'REQUIRE_GUEST_PHONE_VERIFICATION', 'LUXURY_FLAG', 'MAX_MINIMUM_NIGHTS',
       'MIN_MINIMUM_NIGHTS', 'MIN_MAXIMUM_NIGHTS', 'MAX_MAXIMUM_NIGHTS',
       'AVG_MINIMUM_NIGHTS', 'AVG_MAXIMUM_NIGHTS', 'CALENDAR_UPDATED',
       'AVAILABILITY_30', 'AVAILABILITY_60', 'AVAILABILITY_90',
       'AVAILABILITY_365', 'HOST_MONTHS', 'HOST_NEIGHBOURHOOD',
       'HOST_RESPONSE_TIME', 'HOST_RESPONSE_RATE', 'HOST_ACCEPTANCE_RATE',
       'HOST_IS_SUPERHOST', 'HOST_LISTINGS_COUNT', 'HOST_ENTIRE_HOMES',
       'HOST_PRIVATE_ROOMS', 'HOST_SHARED_ROOMS', 'HOST_VERIFICATIONS',
       'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED', 'HOST_ID',
       'NUMBER_OF_REVIEWS', 'MONTH_SINCE_FIRST_REVIEW',
       'MONTH_S

In [7]:
# select columns
col_select = [
    # list
    'ID', 'YEAR_MONTH', 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES'
    , 'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'SQUARE_FEET'
    , 'INSTANT_BOOKABLE', 'GUESTS_INCLUDED', 'CANCELLATION_POLICY'
    , 'REQUIRE_GUEST_PROFILE_PICTURE', 'REQUIRE_GUEST_PHONE_VERIFICATION'
    , 'LUXURY_FLAG'
    # cal
    , 'AVG_MINIMUM_NIGHTS', 'AVG_MAXIMUM_NIGHTS'
    , 'AVAILABILITY_30','AVAILABILITY_60', 'AVAILABILITY_90', 'AVAILABILITY_365'
    # host
    , 'HOST_MONTHS', 'HOST_RESPONSE_RATE','HOST_RESPONSE_TIME'
    , 'HOST_ACCEPTANCE_RATE', 'HOST_IS_SUPERHOST', 'HOST_LISTINGS_COUNT'
    , 'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED'
    , 'HOST_ID'
    # review
    , 'NUMBER_OF_REVIEWS', 'MONTH_SINCE_FIRST_REVIEW'
    , 'MONTH_SINCE_LAST_REVIEW', 'REVIEW_SCORES_RATING', 'REVIEW_SCORES_ACCURACY'
    , 'REVIEW_SCORES_CLEANLINESS', 'REVIEW_SCORES_CHECKIN'
    , 'REVIEW_SCORES_COMMUNICATION', 'REVIEW_SCORES_LOCATION'
    , 'REVIEW_SCORES_VALUE', 'REVIEWS_PER_MONTH'
    # booked
    , 'BOOKED'
    # loation
    , 'NEIGHBOURHOOD', 'IS_LOCATION_EXACT'
    ,'LATITUDE', 'LONGITUDE'
    # price
    , 'TXN_PRICE', 'PRICE_PER_GUEST', 'SECURITY_DEPOSIT', 'CLEANING_FEE', 'EXTRA_PEOPLE'
]
df_listing = df_data[col_select]

In [8]:
# cleaning
df_listing['ROOM_TYPE']=df_listing['ROOM_TYPE'].str.replace('hotel room','private room')

C:\Users\tyler\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
# to csv 
%cd C:\Users\tyler\repos\rshinybnb
    
df_listing.to_csv('data\listing20_01to06.csv',index=False)

C:\Users\tyler\repos\rshinybnb


In [10]:
import pandas as pd
df = pd.read_csv('data\listing20_01to06.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 52 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID                                148835 non-null  int64  
 1   YEAR_MONTH                        148835 non-null  object 
 2   PROPERTY_TYPE                     148835 non-null  object 
 3   ROOM_TYPE                         148835 non-null  object 
 4   ACCOMMODATES                      148835 non-null  int64  
 5   BATHROOMS                         148718 non-null  float64
 6   BEDROOMS                          148606 non-null  float64
 7   BEDS                              147612 non-null  float64
 8   BED_TYPE                          148835 non-null  object 
 9   SQUARE_FEET                       1044 non-null    float64
 10  INSTANT_BOOKABLE                  148835 non-null  int64  
 11  GUESTS_INCLUDED                   148835 non-null  i